## Análisis de tipos de producto CvLAC

In [1]:
import pandas as pd
import requests
import time
import json
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str,
'COD_RH_CREA':str
 }

In [2]:
#cp product_type.json product_type_bak.json

In [39]:
f=open('product_type.json','r')
df=json.load(f)
f.close()

In [41]:
#df=df[:176400]
#f=open('product_type.json','w')
#json.dump(df,f)
#f.close()

In [42]:
len(df)

176400

### Extraer niveles

y convertir a Pandas

In [43]:
ll=[]
for i in range(len(df)):
    d=df[i]
    dd={}
    dd['SGL_CATEGORIA']=d.get('SGL_CATEGORIA')
    while d.get('product_type'):
        if i%100==0:
            print(i,end='\r')
        #print(d.get('product_type'))
        d=d.get('product_type')[0]
        dd[f'NIVEL_{d.get("NRO_NIVEL")}']=f'{d.get("COD_TIPO_PRODUCTO")}: {d.get("TXT_NME_TIPO_PRODUCTO")}'
    if dd:
        ll.append(dd)

convertir a pandas DataFrame

In [44]:
pf=pd.DataFrame(ll)

In [45]:
pf=pf[['SGL_CATEGORIA',  'NIVEL_0', 'NIVEL_1', 'NIVEL_2', 'NIVEL_3']]

### Añadir nivel máximo

In [46]:
def get_nivel_max(row,labels=['Indefinido', 'Otro','Otra']):
    for l in list(range(4))[::-1]:
        if row.get(f'NIVEL_{l}').split(': ')[-1].strip() not in labels:
            NIVEL_MAX=row.get(f'NIVEL_{l}')
            break
    return NIVEL_MAX

In [47]:
pf.shape

(176400, 5)

In [49]:
pf['NIVEL_MAX']=pf.fillna('Indefinido').apply(lambda row: get_nivel_max(row,labels=['Indefinido']),axis='columns')

### Contar niveles

In [50]:
len(pf['NIVEL_0'].unique())

8

In [51]:
len(pf['NIVEL_1'].unique())

63

In [52]:
len(pf['NIVEL_2'].unique())

153

In [53]:
len(pf['NIVEL_3'].unique())

6

In [54]:
len(pf['NIVEL_MAX'].unique())

188

In [55]:
len(pf['SGL_CATEGORIA'].unique())

107

### Extraer tipos de productos únicos

y ordenar por `'SGL_CATEGORIA'` y `'NIVEL_MAX'` para seleccionar ejemplos de valores no vacíos de `'SGL_CATEGORIA'`

In [56]:
pfu=pf.drop_duplicates().sort_values(['SGL_CATEGORIA','NIVEL_MAX']).drop_duplicates().reset_index(drop=True)#.fillna('')

In [57]:
pfu['CATEGORIAS']=pfu['NIVEL_MAX'].apply(lambda s: 
   pfu[pfu.fillna(False)['NIVEL_MAX']==s]['SGL_CATEGORIA'].dropna().str.split('_').str[0
                    ].unique()                      
                     ).apply(lambda l:[ x for x in l if x])

### Visualizar resultado

In [58]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth',200)

In [62]:
pfu=pfu.drop_duplicates(subset='NIVEL_MAX').fillna('').sort_values(['NIVEL_0','NIVEL_1','NIVEL_2']).reset_index(drop=True)
pfu

,SGL_CATEGORIA,NIVEL_0,NIVEL_1,NIVEL_2,NIVEL_3,NIVEL_MAX,CATEGORIAS
0,ART-00,1: Producción bibliográfica,11: Artículo,111: Publicado en revista especializada,,111: Publicado en revista especializada,"[ART-00, ART-ART, ART-GC, PE-00, PE-PE, PF-00, PF-EX, PF-PF, PID-00, PID-EX, PID-PID, PPC-PPC]"
1,ART-00,1: Producción bibliográfica,11: Artículo,112: Corto (Resumen),,112: Corto (Resumen),"[ART-00, ART-ART, ART-GC, PE-00, PE-PE, PF-00, PF-EX, PF-PF, PID-00, PID-PID]"
2,ART-00,1: Producción bibliográfica,11: Artículo,113: Revisión (Survey),,113: Revisión (Survey),"[ART-00, ART-ART, ART-GC, PE-PE, PF-00, PID-00, PID-PID]"
3,ART-00,1: Producción bibliográfica,11: Artículo,114: Caso clínico,,114: Caso clínico,"[ART-00, ART-ART, ART-GC, PE-PE, PF-00, PID-00, PID-PID]"
4,,1: Producción bibliográfica,12: Trabajos en eventos (Capítulos de memoria),121: Completo,,121: Completo,[]
5,,1: Producción bibliográfica,12: Trabajos en eventos (Capítulos de memoria),122: Resumen,,122: Resumen,[]
6,PE-PE,1: Producción bibliográfica,13: Libro,131: Otro libro publicado,,131: Otro libro publicado,"[PE-PE, PF-00, PF-EX, PID-00, PID-PID]"
7,PE-PE,1: Producción bibliográfica,13: Libro,132: Capítulo de libro,,132: Capítulo de libro,"[PE-PE, PF-00, PF-EX, PF-PF, PID-00, PID-PID]"
8,PE-PE,1: Producción bibliográfica,13: Libro,133: Libro pedagógico y/o de divulgación,,133: Libro pedagógico y/o de divulgación,"[PE-PE, PF-00, PID-00, PID-PID]"
9,PE-PE,1: Producción bibliográfica,13: Libro,134: Libro resultado de investigación,,134: Libro resultado de investigación,"[PE-PE, PF-00, PF-EX, PID-00, PID-PID]"


In [63]:
pfu.to_excel('productos_scienti.xlsx',index=False)